In [32]:
import numpy as np

In [33]:
def lp_matrix(x:np.ndarray, p=1) -> float:
    if p == "inf":
        return np.max(np.abs(x), axis = 1)
    return np.sum(np.abs(x)**p, axis = 1)**(1/p)

def lp_vector(x:np.ndarray, p=1) -> float:
    if p == "inf":
        return np.max(np.abs(x))
    return np.sum(np.abs(x)**p)**(1/p)

In [49]:
def numeric_approach_lp_matrix_norm(matrix, p, samples):
    dimension = len(matrix)
    montecarlo = np.random.random((samples, dimension))
    
    #normalize
    montecarlo_norm = montecarlo/lp_matrix(montecarlo, p).reshape(-1, 1)

    norm_induced = lp_matrix(np.dot(matrix, montecarlo_norm.T).T,p)
    idx = np.argmax(norm_induced)

    if p != "inf":
        print("Numeric approach: ", lp_vector(np.dot(matrix,montecarlo_norm[idx,:]),p), " Vs. Real value: ", np.linalg.norm(matrix, ord=p))
    else:
        print("Numeric approach: ", lp_vector(np.dot(matrix,montecarlo_norm[idx,:]),p), " Vs. Real value: ", np.linalg.norm(matrix, ord=np.inf))
    
    return montecarlo_norm[idx,:]

In [98]:
def iterative_numeric_approach_lp_matrix_norm(A, p, samples, max_iter = 500):
    dimension = len(A)
    if p != "inf":
        real_val = np.linalg.norm(A, ord=p)
    else:
        real_val = np.linalg.norm(A, ord=np.inf)
    
    montecarlo = np.random.random((samples*10, dimension))

    #normalize
    montecarlo_norm = montecarlo/lp_matrix(montecarlo, p).reshape(-1, 1)

    norm_induced = lp_matrix(np.dot(A, montecarlo_norm.T).T,p)
    idx = (-norm_induced).argsort()[:2]

    vector_a = montecarlo_norm[idx[0],:]
    vector_b = montecarlo_norm[idx[1],:]


    for j in range(max_iter):

        points_alpha = np.random.random((dimension, samples)); points_beta = np.random.random((dimension, samples))

        combination = np.array([np.multiply(points_alpha[:,i],vector_a) + np.multiply(1-points_alpha[:,i],vector_b) for i in range(samples)]).reshape(-1, dimension)
        combination_norm = combination/lp_matrix(combination, p).reshape(-1, 1)
        
        norm_induced = lp_matrix(np.dot(A, combination_norm.T),p)
        idx = (-norm_induced).argsort()[:2]

        if np.abs(lp_vector(np.dot(A,combination_norm[idx[0],:]),p) - real_val) < 0.01:
            print(j)
            print("Numeric approach: ", lp_vector(np.dot(A,combination_norm[idx[0],:]),p), " Vs. Real value: ", real_val)
            return combination_norm[idx[0],:]
        else:
            if np.abs(lp_vector(np.dot(A,vector_a),p) - real_val) < np.abs(lp_vector(np.dot(A,combination_norm[idx[0],:]),p) - real_val):
                vector_a = combination_norm[idx[0],:]
                vector_b = combination_norm[idx[1],:]
    
    print(j)
    print("Numeric approach: ", lp_vector(np.dot(A,combination_norm[idx[0],:]),p), " Vs. Real value: ", real_val)
    return combination_norm[idx[0],:]



In [103]:
A = np.random.randint(-10,10,(5,5))
p = 2
print(A)

print("vector method 1: ", numeric_approach_lp_matrix_norm(A, p, 1000))

print("Vector method iterative: ", iterative_numeric_approach_lp_matrix_norm(A, p, 1000, 1000))

[[-10   5   4  -6   3]
 [ -1  -4   1  -5   1]
 [  8   4  -7   4   7]
 [  1  -6  -7  -2   4]
 [  4  -9  -3   5  -8]]
Numeric approach:  15.201910506575206  Vs. Real value:  19.501959001460722
vector method 1:  [0.70764548 0.10402994 0.05563279 0.69562742 0.03772447]
999
Numeric approach:  15.492633722521848  Vs. Real value:  19.501959001460722
Vector method iterative:  [0.04227157 0.811923   0.05515856 0.02119786 0.57922564]


In [101]:
np.sum(A,axis=0)

array([  2,   3, -20,  -1, -21])